# Introduction

## Purpose:

- __To verify that the addition of a new constraint $F_k$ to a set of constraints $\{F_{\text{1 , ... , j}}\}$ allows the inference of new formulas, otherwise it is the symptom that this new constraint is redundant. Which means $F_k$ is not inferred by orginial set $\{F_1,...,F_j \}$__

## Definition:

- __Let $\{F_1, ..., F_j\}$ be a set of constraints. Let $F_k$ be a new instruction. If $F_1\land ...\land F_j\land F_k\not\models\bot$, *i.e, $\{F_1, ..., F_j, F_k\}$ is satisfiable*, then $F_k$ is not redundant iff $F_1 \land ... \land F_j \land \neg F_k \not\models\bot$ *i.e, $\{F_1, ..., F_j, \neg F_k\}$ is satisfiable*.__

## Conclusion:

- __Steps to check whether a new constraint $F_k$ is redundant to a existing or not.__

- __1) Add $F_k$ to existing set $\{F_{\text{1 , ... , j}}\}$, check the satisfiability;__

- __2) Remove $F_k$ and add $\neg F_k$, then, check the satisfiability;__

- __3) If both operations guarantee the satisfiability, the new constraint $F_k$ is non-redundant.__

# Implementation with Z3Py

In [1]:
from z3 import *
import re

In [2]:
def redunCheck(s, newCond):
    '''
    Description: pass through a set of constrains s,
    check the redundecny of new constraint newCond
    :type z3.z3.Solver: s
    :type z3.z3.BoolRef: newCond
    :rtype: string
    '''
    res = ''
    # Add the new constriant to existing set
    s.push() 
    s.add(newCond)
    r1 = (s.check()==sat)
    # Remove the new constraint
    s.pop()
    # Add the negation of new constraint 
    s.push()
    s.add(Not(newCond))
    r2 = (s.check()==sat)
    # Remove the constraint just added
    s.pop()
    # Check the redundency
    if r1 == True and r2 == True:
        res = 'Non-redundant'
    else:
        res = 'Redundant'
    return res

## Example 1 - Arithmetic Operations

- __1) Variables: x as int, y as int__

- __2) Existing Statement:__

    - __$i: (x+y)^2 > 10, x>10, y>10$__

- __3) New Constraints:__

    - __$\text{New Constraint 1: } x > 0$__

    - __$\text{New Constraint 2: } x < 100$__

In [3]:
# Part 1: Defince constraints
# Define Variables
x = Int('x')
y = Int('y')

In [4]:
# Create an empty constraints set
s = Solver()

In [5]:
# Define the function t = (x+y)^2
t = simplify((x + y)**2, som=True)

In [6]:
# Add first constraint to body
s.push()

In [7]:
s.add(t>10)

In [8]:
# Display the body
s.assertions()

[x*x + 2*x*y + y*y > 10]

In [9]:
# Add other constraints, x > 10, y > 10
s.push()

In [10]:
s.add(x>10,y>10)

In [11]:
# Display the body
s.assertions()

[x*x + 2*x*y + y*y > 10, x > 10, y > 10]

In [12]:
# Part 2: Check new constraints
# Define first new constraint, x > 0
f = x > 0

In [13]:
# Check the redundency
redunCheck(s, f)

'Redundant'

- __Explain: The existing constraint x > 10 always guarantee x > 0, so the new constraint x > 0 is redundant__

In [14]:
s.assertions()

[x*x + 2*x*y + y*y > 10, x > 10, y > 10]

In [15]:
# Define first new constraint, x > 0
g = x < 100

In [16]:
# Check the redundency
redunCheck(s, g)

'Non-redundant'

- __Explain: The new constraint x < 100 cannot be infered from the exisiting constraints, so it is Non-redundant__

## Example 2 - Machine Arithmetic

In [17]:
x = BitVec('x', 16)
y = BitVec('y', 16)

In [18]:
s = Solver()

In [19]:
t = (x + y == 10)

In [20]:
s.push()

In [21]:
s.add(t)

In [22]:
s.assertions()

[x + y == 10]

In [23]:
s.push()

In [24]:
s.add(x>=3,y>=3)

In [25]:
s.assertions()

[x + y == 10, x >= 3, y >= 3]

In [26]:
f = x > 1

In [27]:
redunCheck(s, f)

'Redundant'

In [28]:
g = x > 3

In [29]:
redunCheck(s, g)

'Non-redundant'

# Implementation with YICES

# Reference:

## [z3py-tutorial](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## [z3 Guide](https://rise4fun.com/z3/tutorial)